In [20]:
import pandas as pd
import random
import csv

def make_tab(file_index):
    # Load the data
    file_path = 'participants-seoul.csv'  # Update this to the correct file path
    df = pd.read_csv(file_path)

    # Extract the necessary columns
    participants = df[['univ', 'member']]

    # Shuffle the participants
    participants_shuffled = participants.sample(frac=1).reset_index(drop=True)

    # Initialize the rooms
    rooms = {i: [] for i in range(1, 9)}

    # Assign participants to rooms ensuring no two from the same university are in the same room
    for index, row in participants_shuffled.iterrows():
        for room_number, room_members in rooms.items():
            if len(room_members) < 4 and not any(row['univ'] == member['univ'] for member in room_members):
                rooms[room_number].append(row.to_dict())
                break

    # Check for unassigned participants
    unassigned = participants_shuffled[~participants_shuffled['member'].isin([member['member'] for room in rooms.values() for member in room])]

    if not unassigned.empty:
        return

    # save the room assignments to a csv file
    with open(f'outputs/room_assignments_{file_index}.csv', 'w') as f:
        f.write('room_number,member,univ\n')
        for room_number, room_members in rooms.items():
            for member in room_members:
                f.write(f"{room_number},{member['member']},{member['univ']}\n")

In [ ]:
for i in range(1, 101):
    make_tab(i)

In [7]:
import pandas as pd
from collections import defaultdict

def check(first_file, second_file):
    # Load the CSV files
    file1 = f'outputs/room_assignments_{first_file}.csv'
    file2 = f'outputs/room_assignments_{second_file}.csv'

    df1 = pd.read_csv(file1)
    df2 = pd.read_csv(file2)

    # Create dictionaries to store room assignments
    room_assignments_1 = defaultdict(set)
    room_assignments_2 = defaultdict(set)

    # Fill the dictionaries with the data
    for _, row in df1.iterrows():
        room_assignments_1[row['room_number']].add(row['member'])

    for _, row in df2.iterrows():
        room_assignments_2[row['room_number']].add(row['member'])

    # Create a dictionary to store the count of meetings for each person
    meetings_count = defaultdict(int)

    # Function to count meetings
    def count_meetings(room_assignments_1, room_assignments_2):
        for room2, members2 in room_assignments_2.items():
            for member in members2:
                for room1, members1 in room_assignments_1.items():
                    if member in members1:
                        common_members = members1.intersection(members2)
                        if len(common_members) > 1:
                            meetings_count[member] += len(common_members) - 1

    # Count meetings between members across the two sets of room assignments
    count_meetings(room_assignments_1, room_assignments_2)

    # Create the output CSV file
    output_df = pd.DataFrame(list(meetings_count.items()), columns=['member', 'meeting_count'])
    output_file = f'outputs/meeting_counts_{first_file}_{second_file}.csv'
    # Print the total count of meetings
    total_meetings = output_df['meeting_count'].sum()
    print(f'Total count of meetings: {total_meetings}')
    # add total count of meetings to the output file
    output_df.loc[len(output_df)] = ['Total', total_meetings]
    output_df.to_csv(output_file, index=False)

    output_df.head()

In [69]:
check(40, 100)

Total count of meetings: 10


- 1-10 : 0
- 1-22 : 4
- 1-40 : 4

# final
- 1-10-40-100


member,meeting_count
김민지,1
이관우,1
박준상,1
김정윤,1
Total,4

member,meeting_count
김채린,1
김성진,1
최지운,1
김지호,1
김대건,1
장동훈,1
Total,6

member,meeting_count
박근형,1
김지호,1
유승현,1
김다산,1
소윤호,1
김대건,1
오주호,1
서준교,1
한윤지,1
김민혁,1
Total,10

member,meeting_count
김민혁,1
김정윤,1
김성진,1
박준상,1
소윤호,1
김대건,1
김민지,1
윤준석,1
Total,8

member,meeting_count
곽규용,1
박지현,1
이관우,1
박준상,1
조봉재,1
김도안,1
소윤호,1
김대건,1
한윤지,1
김은규,1
Total,10
